pykrx를 통하여 수집한 종목 마스터 테이블을 활용하여 이제 각 종목을 시각화 해보자. 필요한 라이브러리를 불러온 후 우리가 수집한 마스터 테이블을 로드해준다.

(앞서 선정한 종목들 중 2차전지의 pykrx의 데이터 중 '에코프로비엠'의 데이터에 오류가 있어 '코스모신소재'로 다시 선정하여 대체하였다.)

# 주식 시세 조회

In [134]:
from pykrx import stock
from pykrx import bond
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

## 주가 마스터 테이블 로드

In [135]:
master = pd.read_csv('master_frame.csv')
master.head(5)

,ticker,corp_name,market
0,095570,AJ네트웍스,KOSPI
1,006840,AK홀딩스,KOSPI
2,027410,BGF,KOSPI
3,282330,BGF리테일,KOSPI
4,138930,BNK금융지주,KOSPI


## 목표

이번 실습의 목표는 업종별 주식 3개와 종목별 비교를 해보는 것이 목표이다. 업종별로는 2차전지와 마이데이터를 선정하였다.

- 2차전지
  - 삼성SDI (006400)
  - LG에너지솔루션 (373220)
  - 코스모신소재 (005070)

- 마이데이터
  - 나이스평가정보 (030190)
  - 파이오링크 (170790)
  - 헥토이노베이션 (214180)
 
각 종목에 대한 ticker는 구글에서 검색하여 구했으며, 앞으로는 DART api를 통하여 merge 후 연관을 시킬 예정이다.

In [177]:
battery_ticker = master[master['ticker'].str.contains('006400|373220|005070')] 
battery_ticker

,ticker,corp_name,market
95,373220,LG에너지솔루션,KOSPI
429,006400,삼성SDI,KOSPI
734,005070,코스모신소재,KOSPI


In [137]:
mydata_ticker = master[master['ticker'].str.contains('030190|170790|214180')] 
mydata_ticker

,ticker,corp_name,market
1026,030190,NICE평가정보,KOSDAQ
2394,170790,파이오링크,KOSDAQ
2549,214180,헥토이노베이션,KOSDAQ


pykrx의 공식문서를 통하여 종목정보를 불러오는 메서드를 가져왔다. 메서드에 해당 ticker를 지정해주면 그 종목의 데이터를 가져와준다. 우선 시험삼아 LGd에너지솔루션의 데이터를 가져와봤다.

In [138]:
LG = stock.get_market_ohlcv("20180101", "20230601", "373220")

In [139]:
LG['ticker'] = '373220'

In [140]:
LG = LG.reset_index()

## 시각화

### LG에너지솔루션
일단 먼저 LG에너지솔루션 데이터를 통해 시각화를 진행해본다. 라이브러리는 plotly를 사용할 것이며, plotly의 공식문서를 참조하였다.

우선 선 차트를 활용하여 LG에너지솔루션의 시가를 시각화해보자.

#### line chart

In [143]:
LG.head(5)

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker
0,2022-01-27,597000,598000,450000,505000,15946992,8155323955500,-15.41,373220
1,2022-01-28,476000,483000,445000,450000,4559773,2104994467000,-10.89,373220
2,2022-02-03,458000,495500,441000,477000,2918435,1375706997000,6.00,373220
3,2022-02-04,476500,505000,476000,504000,2088996,1038186622000,5.66,373220
4,2022-02-07,520000,548000,511000,548000,1911176,1020155389000,8.73,373220


In [148]:
fig = px.line(LG, x='날짜', y='시가', title='LG에너지솔루션 시가')


fig.update_xaxes(
    rangeslider_visible=True,     #rangeslider를 지정 : 원하는 범위의 그래프만 볼 수 있다.
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.update_layout(
    hovermode="x unified",
    plot_bgcolor="white"
)
fig.show()

다음으로 LG에너지솔루션의 거래량을 확인해보자. 거래량은 바 차트를 활용한다.

#### bar chart

In [147]:
fig = px.bar(LG, x='날짜', y='거래량', title='LG에너지솔루션 거래량')

fig.update_layout(
    hovermode="x unified",
    plot_bgcolor="white"
)

fig.show()

위에서 만든 두 그래프를 혼합하여 한꺼번에 나타내보자.

#### line&bar chart 혼합

In [191]:
emd_area = 'LG에너지솔루션'

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        name = '거래량',
        x = LG['날짜'],
        y = LG['거래량']
    )
)

fig.add_trace(
    go.Scatter(
        name = '시가',
        x = LG['날짜'],
        y = LG['시가'],
        yaxis="y2"
    )
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)


fig.update_layout(
    title= 'LG에너지솔루션 거래량 및 거래금액 <br><sup>단위(만원)</sup>',
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='white',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_white',
    xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y=1.1), #Adjust legend position
    barmode='group'
)

LG에너지솔루션의 시각화된 차트를 확인해보면 우선 LG에너지솔루션의 상장 당일 가격이 고점에서 시작했다가 상장일 이후로 점차 하락하는 것을 확인할 수 있다. 그 이후 약 3개월간 큰 박스권에서 등락을 반복하다가 2022년 7월 이후로 큰 폭의 상승을 한 것을 볼 수 있다. 그 이후 2022년 연말 큰 폭의 하락을 다시 겪었다가 2023년 초 부터 현재까지 다시 하락분을 잡아먹는 큰 상승을 해오고 있다.
또한 LG에너지솔루션은 상장 당일 역대 가장 큰 거래량을 기록한 이후 그 거래량을 넘어서진 못하고 있다. 상장 당일의 거래량이 워낙 컸기에 상대적으로 그 이후의 거래량이 작아보이는 경향이 있다.

### 나이스평가정보

LG에너지솔루션을 시각화 해봤으니, 이번엔 나이스평가정보를 시각화해보자. 기존 코드에서는 하드코딩이 되어 있어서 LG에너지솔루션의 변수는 LG로 지정했고, 타이틀마저 하드코딩 되어 있기 때문에 이번에는 그 부분을 고려하여 시각화해보자.

In [186]:
now = datetime.datetime.now()    #변수의 유연성을 부여하기 위해 데이트타임 라이브러리를 활용할것이다.

print (now.strftime("%Y/%m(%B)/%d %A %p %I:%m:%S, 일년 중 %U 번째주, 일년 중 %j번째 날 "))
print(now.strftime("%Y%m%d"))

2023/06(June)/01 Thursday PM 01:06:37, 일년 중 22 번째주, 일년 중 152번째 날 
20230601


In [187]:
start_date = "20190101"
end_date = now.strftime("%Y%m%d")
ticker = "030190"
ticker_nm = master[master['ticker'] == ticker].reset_index(drop=True)['corp_name'][0]

In [188]:
stock_data = stock.get_market_ohlcv(start_date, end_date, ticker)
stock_data['ticker'] = ticker
stock_data = stock_data.reset_index()

In [189]:
stock_data.head(3)

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker
0,2019-01-02,10900,10950,10250,10500,150186,1578381600,-4.11,030190
1,2019-01-03,10550,10650,10250,10400,92220,960370150,-0.95,030190
2,2019-01-04,10400,10550,10250,10500,107643,1126248750,0.96,030190


In [190]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        name = '거래량',
        x = stock_data['날짜'],
        y = stock_data['거래량'],
        #marker = {'color':'black'}
    )
)

fig.add_trace(
    go.Scatter(
        name = '시가',
        x = stock_data['날짜'],
        y = stock_data['시가'],
        #marker = {'color': 'black'},
        yaxis="y2"
    )
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)


fig.update_layout(
    #title= '나이스평가정보 거래량 및 거래금액 <br><sup>단위(만원)</sup>',
    title= f'{ticker_nm} 거래량 및 시가',
    
    #title_font_family="맑은고딕",
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='white',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_white',
    xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y=1.1), #Adjust legend position
    barmode='group'
)

나이스평가정보의 경우 타 종목들과 마찬가지로 2020년 3월경에 코로나 특수로 인한 큰 폭의 하락을 겪은 후 2021년 초까지 상승을 보이다가 현재까지 하락하는 추세를 보이고 있다. (위의 그래프에서 거래량의 경우 타임프레임이 커서 상대적으로 얕은 음영을 보이고 있다.)

## 같은 업종의 주가 비교

### 이차전지

지금껏 LG에너지솔루션과 나이스평가정보의 주가 데이터를 시각화 해보았다. 이번에는 LG에너지솔루션의 업종인 2차전지의 대표 종목 주가데이터를 파악해보자.

In [178]:
battery_ticker_list = ['006400','373220','005070']

In [179]:
start_date = "20190101"
end_date = now.strftime("%Y%m%d")

stock_total_data = pd.DataFrame()

for ticker in battery_ticker_list:
    print(ticker)
    ticker_nm = master[master['ticker'] == ticker].reset_index(drop=True)['corp_name'][0]
    
    stock_data = stock.get_market_ohlcv(start_date, end_date, ticker)
    stock_data['ticker'] = ticker
    stock_data = stock_data.reset_index()
    
    stock_total_data = pd.concat([stock_total_data, stock_data])

006400
373220
005070


In [180]:
stock_total_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker
0,2019-01-02,222000,224000,208500,210500,345548,73769630500,-3.88,006400
1,2019-01-03,209000,209500,202000,203000,484354,99081075000,-3.56,006400
2,2019-01-04,201500,204000,194000,201000,474274,94114426500,-0.99,006400
3,2019-01-07,209000,219500,208500,217000,549913,118366945500,7.96,006400
4,2019-01-08,218000,219500,213000,218500,310061,67281043682,0.69,006400
...,...,...,...,...,...,...,...,...,...
1086,2023-05-25,187400,188300,181100,181300,368147,67909901500,-3.31,005070
1087,2023-05-26,180700,184100,173100,174800,466141,82916240000,-3.59,005070
1088,2023-05-30,176000,181000,163500,173300,761071,130289951000,-0.86,005070
1089,2023-05-31,173300,178700,169500,178700,2396165,424943749800,3.12,005070


In [181]:
stock_total_data = pd.merge(stock_total_data, master, 
        how='left',
        on='ticker')

In [182]:
stock_total_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker,corp_name,market
0,2019-01-02,222000,224000,208500,210500,345548,73769630500,-3.88,006400,삼성SDI,KOSPI
1,2019-01-03,209000,209500,202000,203000,484354,99081075000,-3.56,006400,삼성SDI,KOSPI
2,2019-01-04,201500,204000,194000,201000,474274,94114426500,-0.99,006400,삼성SDI,KOSPI
3,2019-01-07,209000,219500,208500,217000,549913,118366945500,7.96,006400,삼성SDI,KOSPI
4,2019-01-08,218000,219500,213000,218500,310061,67281043682,0.69,006400,삼성SDI,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...
2508,2023-05-25,187400,188300,181100,181300,368147,67909901500,-3.31,005070,코스모신소재,KOSPI
2509,2023-05-26,180700,184100,173100,174800,466141,82916240000,-3.59,005070,코스모신소재,KOSPI
2510,2023-05-30,176000,181000,163500,173300,761071,130289951000,-0.86,005070,코스모신소재,KOSPI
2511,2023-05-31,173300,178700,169500,178700,2396165,424943749800,3.12,005070,코스모신소재,KOSPI


In [183]:
fig = px.line(stock_total_data, x="날짜", y="시가", color='corp_name')



# fig.update_xaxes(
#     rangeslider_visible=True,
#     rangeselector=dict(
#         buttons=list([
#             dict(count=1, label="1m", step="month", stepmode="backward"),
#             dict(count=6, label="6m", step="month", stepmode="backward"),
#             dict(count=1, label="YTD", step="year", stepmode="todate"),
#             dict(count=1, label="1y", step="year", stepmode="backward"),
#             dict(step="all")
#         ])
#     )
# )


fig.update_layout(
#     title= '엘지에너지솔루션 거래량 및 거래금액 <br><sup>단위(만원)</sup>',
    title= f'2차전지 대표종목 비교',
    
    #title_font_family="맑은고딕",
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='white',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_white',
#     xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y=1.1), #Adjust legend position
    barmode='group'
)

fig.show()

In [195]:
cosmo = pd.DataFrame()
cosmo = stock_total_data[stock_total_data['ticker']=='005070']
fig = px.line(cosmo, x='날짜', y='시가', title='코스모신소재 시가')


fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.update_layout(
    hovermode="x unified",
    plot_bgcolor="white",
    yaxis_tickformat = ','
)
fig.show()

삼성SDI의 경우 코로나 발발 시점에 단기적인 큰 하락 겪은 후 2021년초까지 큰 폭의 상승을 보여주다가 2021년~2022년초까지 약 1년간 큰 박스권 내에서 등락을 반복했다. 그 이후 박스권을 벗어난 하락을 보여주다가 현재까지 점차 상승하는 추세를 나타내고 있다. LG에너지솔루션의 경우도 상장 이후로 큰 하락을 보여준 다음에 삼성SDI와 같은 그래프 형태를 그리며 상승 추세를 보여주고 있다. 
특별히 코스모신소재의 경우 나머지 두 종목의 가격프레임과 차이가 커 세 종목을 한꺼번에 표현하면 자세히 관찰하기가 힘들어 따로 시각화하여 나타내주었다. 코스모신소재는 다른 두 종목들과 다르게 코로나 이전과 이후로도 상대적으로 큰 등락을 보이진 않았다. 그러나 2023년 초부터 현재까지 엄청나게 큰 폭의 상승을 보여주며 2차전지 강세장의 대표종목으로 우뚝 서고 있다.

### 마이데이터

이번에는 마이데이터 업종의 대표종목을 비교해보자.

In [204]:
mydata_ticker_list = ['030190','170790','214180']

In [209]:
start_date = "20190101"
end_date = now.strftime("%Y%m%d")

stock_total_data = pd.DataFrame()

for ticker in mydata_ticker_list:
    print(ticker)
    ticker_nm = master[master['ticker'] == ticker].reset_index(drop=True)['corp_name'][0]
    
    stock_data = stock.get_market_ohlcv(start_date, end_date, ticker)
    stock_data['ticker'] = ticker
    stock_data = stock_data.reset_index()
    
    stock_total_data = pd.concat([stock_total_data, stock_data])

030190
170790
214180


In [210]:
stock_total_data = pd.merge(stock_total_data, master, 
        how='left',
        on='ticker')

In [211]:
stock_total_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker,corp_name,market
0,2019-01-02,10900,10950,10250,10500,150186,1578381600,-4.11,030190,NICE평가정보,KOSDAQ
1,2019-01-03,10550,10650,10250,10400,92220,960370150,-0.95,030190,NICE평가정보,KOSDAQ
2,2019-01-04,10400,10550,10250,10500,107643,1126248750,0.96,030190,NICE평가정보,KOSDAQ
3,2019-01-07,10600,10650,10400,10600,102352,1078504450,0.95,030190,NICE평가정보,KOSDAQ
4,2019-01-08,10650,10700,10400,10400,59433,621882450,-1.89,030190,NICE평가정보,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...
3268,2023-05-25,12640,12710,12560,12690,28419,358781440,0.00,214180,헥토이노베이션,KOSDAQ
3269,2023-05-26,12630,12670,12520,12590,30833,387628740,-0.79,214180,헥토이노베이션,KOSDAQ
3270,2023-05-30,12590,12830,12380,12740,88124,1118660030,1.19,214180,헥토이노베이션,KOSDAQ
3271,2023-05-31,12670,12870,12620,12650,27268,346320110,-0.71,214180,헥토이노베이션,KOSDAQ


In [212]:
fig = px.line(stock_total_data, x="날짜", y="시가", color='corp_name')



fig.update_layout(
    title= f'마이데이터 대표종목 비교',
    
    #title_font_family="맑은고딕",
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='white',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_white',
#     xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y=1.1), #Adjust legend position
    barmode='group'
)

fig.show()

2019년 하반기 이전으로는 세 종목이 비슷한 움직임을 보이다가 2019년 하반기 부터 헥토이노베이션은 먼저 하락을 보이기 시작했다. 그 이후 코로나 발생 시점 세 종목 모두 큰 폭의 하락을 보여주고 이후 2021년 중순까지 다같이 재차 상승하는 양상을 보여줬다. 그러나 2021년 초 ~ 2021년 중순의 경우 나이스평가 정보와 헥토이노베이션이 비교적 큰 상승 추세를 보이는 반면에 파이오링크는 상승 보다는 횡보의 모습을 보여줬다. 2021년 하반기 이후에는 마이데이터의 세 종목 모두가 큰 하락 형태를 나타냈다. 그러나 파이오링크의 경우 2022년 말부터 현재까지 다른 두 종목에 비해 비교적 강한 반등세를 보이며 주가를 회복하고 있다. 
마이데이터 업종의 경우 업종 테마에 묶인 움직임을 보이기도 하지만 각각의 개별 종목이 업종의 큰 움직임에서 벗어난 움직임을 보여주기도 한다.

## PER, PBR

이번에는 마이데이터 대표 종목들의 PER,PBR 데이터를 불러와보자.

In [213]:
start_date = "20190101"
end_date = now.strftime("%Y%m%d")

per_pbr_total_data = pd.DataFrame()

for ticker in mydata_ticker_list:
    print(ticker)
    ticker_nm = master[master['ticker'] == ticker].reset_index(drop=True)['corp_name'][0]
    
    stock_data = stock.get_market_fundamental(start_date, end_date, ticker)
    stock_data['ticker'] = ticker
    stock_data = stock_data.reset_index()
    
    per_pbr_total_data = pd.concat([per_pbr_total_data, stock_data])

030190
170790
214180


In [214]:
stock_total_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker,corp_name,market
0,2019-01-02,10900,10950,10250,10500,150186,1578381600,-4.11,030190,NICE평가정보,KOSDAQ
1,2019-01-03,10550,10650,10250,10400,92220,960370150,-0.95,030190,NICE평가정보,KOSDAQ
2,2019-01-04,10400,10550,10250,10500,107643,1126248750,0.96,030190,NICE평가정보,KOSDAQ
3,2019-01-07,10600,10650,10400,10600,102352,1078504450,0.95,030190,NICE평가정보,KOSDAQ
4,2019-01-08,10650,10700,10400,10400,59433,621882450,-1.89,030190,NICE평가정보,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...
3268,2023-05-25,12640,12710,12560,12690,28419,358781440,0.00,214180,헥토이노베이션,KOSDAQ
3269,2023-05-26,12630,12670,12520,12590,30833,387628740,-0.79,214180,헥토이노베이션,KOSDAQ
3270,2023-05-30,12590,12830,12380,12740,88124,1118660030,1.19,214180,헥토이노베이션,KOSDAQ
3271,2023-05-31,12670,12870,12620,12650,27268,346320110,-0.71,214180,헥토이노베이션,KOSDAQ


In [215]:
per_pbr_total_data = pd.merge(per_pbr_total_data, master, 
        how='left',
        on='ticker')

In [216]:
stock_total_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker,corp_name,market
0,2019-01-02,10900,10950,10250,10500,150186,1578381600,-4.11,030190,NICE평가정보,KOSDAQ
1,2019-01-03,10550,10650,10250,10400,92220,960370150,-0.95,030190,NICE평가정보,KOSDAQ
2,2019-01-04,10400,10550,10250,10500,107643,1126248750,0.96,030190,NICE평가정보,KOSDAQ
3,2019-01-07,10600,10650,10400,10600,102352,1078504450,0.95,030190,NICE평가정보,KOSDAQ
4,2019-01-08,10650,10700,10400,10400,59433,621882450,-1.89,030190,NICE평가정보,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...
3268,2023-05-25,12640,12710,12560,12690,28419,358781440,0.00,214180,헥토이노베이션,KOSDAQ
3269,2023-05-26,12630,12670,12520,12590,30833,387628740,-0.79,214180,헥토이노베이션,KOSDAQ
3270,2023-05-30,12590,12830,12380,12740,88124,1118660030,1.19,214180,헥토이노베이션,KOSDAQ
3271,2023-05-31,12670,12870,12620,12650,27268,346320110,-0.71,214180,헥토이노베이션,KOSDAQ


In [217]:
fig = px.line(per_pbr_total_data, x="날짜", y="PER", color='corp_name')



fig.update_layout(
    title= f'마이데이터 대표종목 PER 비교',
    
    #title_font_family="맑은고딕",
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='white',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_white',
#     xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y=1.1), #Adjust legend position
    barmode='group'
)

fig.show()

In [218]:
fig = px.line(per_pbr_total_data, x="날짜", y="PBR", color='corp_name')



fig.update_layout(
    title= f'마이데이터 대표종목 PBR 비교',
    
    #title_font_family="맑은고딕",
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='white',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_white',
#     xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y=1.1), #Adjust legend position
    barmode='group'
)

fig.show()

위에서 얻어낸 PER,PBR 데이터와 종목들의 시가 데이터를 한꺼번에 나타내어 보자.

In [220]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

df = px.data.stocks().set_index('date')

fig1 = px.line(df[['GOOG', 'AAPL']])
fig2 = px.line(df[['AMZN', 'MSFT']])

fig1 = px.line(stock_total_data, x="날짜", y="시가", color='corp_name', title='시가')
fig2 = px.line(per_pbr_total_data, x="날짜", y="PER", color='corp_name', title='PER')
fig3 = px.line(per_pbr_total_data, x="날짜", y="PBR", color='corp_name', title='PBR')


fig = make_subplots(rows=3, cols=1,
    subplot_titles=("시가", "PER", "PBR"))

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig1['data'][1], row=1, col=1)
fig.add_trace(fig1['data'][2], row=1, col=1)

fig.add_trace(fig2['data'][0], row=2, col=1)
fig.add_trace(fig2['data'][1], row=2, col=1)
fig.add_trace(fig2['data'][2], row=2, col=1)

fig.add_trace(fig3['data'][0], row=3, col=1)
fig.add_trace(fig3['data'][1], row=3, col=1)
fig.add_trace(fig3['data'][2], row=3, col=1)

# fig.add_trace(fig2['data'], row=2, col=1)
# fig.add_trace(fig2['data'][1], row=2, col=1)
# fig.add_trace(fig3['data'], row=3, col=1)
# fig.add_trace(fig3['data'][1], row=3, col=1)

    
fig.show()


PER과 PBR은 낮을수록 저평가되어 있는 주식이라고 흔히들 얘기한다. 위의 시각화된 그래프를 통해 보자면 나이스평가정보에 비해 파이오링크와 헥토이노베이션이 상대적으로 저평가되어 있는 종목이라고 유추해볼 수 있다. 특히 두드러지는 점은 2020년 중반의 파이오링크 PER인데, 자세한 원인 몰라도 그 특정 시점이후 파이오링크의 PER이 파격적으로 감소하였다. 파이오링크는 그 이전까지는 상당히 고평가된 주식 종목으로 볼 수 있었으나 그 시점 이후로 현재까지 상대적으로 안정적인 PER과 PBR을 유지하고 있다.
나이스 평가정보의 경우 상대적으로 높은 PBR 값을 유지해오며 높은 주가를 유지해오다가 주가가 낮아짐과 동시에 PBR 수치도 내려가고 있다. 주가의 영향인지 몰라도 현재로써는 과거에 비해 나이스평가정보의 PBR 값은 안정적인 추세로 가고 있다고 볼 수 있다.